# Exemplary usage of TitanicClassifier

### Initiate environment

In [1]:
using Pkg
Pkg.activate(".")

using DataFrames
using Plots
using TitanicClassifier

  Activating project at `~/CTU/SEM_5/JUL/TitanicClassifier/examples`


In [3]:
c = Dict(A => "kok")
print(c["A"])

UndefVarError: UndefVarError: A not defined